In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import random as sparse_random
from sklearn.random_projection import sparse_random_matrix
from sklearn.decomposition import TruncatedSVD
import keras 
from keras.models import Sequential 
from keras.layers import Dense
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#X = sparse_random(100, 100, density=0.01, format='csr',random_state=42)
#svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
#svd.fit(X)
dataset  = ["movies",
            "ratings",
            "users"]
header_movies = ["movie_id,movie,genres"]
header_ratings = ["user_id","movie_id","rating","timestamp"]
header_users = ["user_id","gender","age","occupation","zip_code"]


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def LeerDatos(filename : str, separa : str, header = True):
    if (header):
        data = pd.read_csv(filename + ".csv", sep =separa, header = 0)
    else:
        data = pd.read_csv(filename+ ".csv", sep = separa, header = None)
    return data.to_numpy()

In [ ]:
def GenerarW( num_capas : int, dim_capas = []):
    W = {}
    for i in range(0,num_capas+1):
        if (i == 0):
            temp = np.random.randn( dim_capas[i], dim_capas[i+1] )
            W[i] = temp
        if (i != 0):
            temp = np.random.randn( dim_capas[i]+1, dim_capas[i+1] )
            W[i] = temp       
    return W

In [ ]:
def Sigmoidal(X:np.array, theta:np.array):
    pot = X.dot(theta)
    return 1/(1+ np.exp(-pot))

In [ ]:
def Forward (X: np.array, W : {}): 
    A = {}
    h_l = X
    A[0] = h_l
    for i in range(0, len(W)):
        if (i == len(W)-1):
            h_l = Sigmoidal(h_l, W[i])
        else:
            h_l = Sigmoidal(h_l, W[i])
            bias = np.ones( (np.size(h_l[:,0]),1) )
            h_l = np.hstack( (bias,h_l) )
     
        A[i+1] = h_l   
    return A

In [ ]:
# Rpuv
def calculate_Rpuv(n,m,a,b,W,U,V,z,p,q):
    Rpuv = np.empty((n,m))
    for u in range(0,n):
        for v in range(0,m):
            #Sumatoria
            sum  = 0
            for ai in range(0,a):
                for bj in range(0,b):
                    sum = sum + W[ai,bj]*U[u,ai]*V[v,bj]
            Rpuv[u,v] = z + p[u] + q[v] + sum
    return Rpuv

In [ ]:

def calculate_DIFuv(R, Rpuv):
    #DIFuv = δuv (R̂uv − Ruv)
    DIFuv = Rpuv - R
    
    n = R.shape[0]
    m = R.shape[1]
    for i in range(0,n):
        for j in range(0,m):
            if R[i,j]==0:
                DIFuv[i,j]= DIFuv[i,j]*0
    
    return DIFuv

In [ ]:
def random_(a,b):
    U = np.random.random((a,b))
    return U

In [ ]:
def Feature_Representation(R,ηp,iter):
    n = R.shape[0]
    m = R.shape[1]

    a = R.shape[1]
    b = R.shape[0]
    η = ηp#np.random.randn() 
    #Randon Initialize z, p, q, U, V, W;
    U = random_(n,a)#np.random.random((n,a))
    V = random_(m,b)

    z = np.random.randn()
    p = np.random.randn(n)
    q = np.random.randn(m)
    W = random_(a,b)

    #Calculate 1 according to Eq.(24); 
        #Rpuv = z + pu + qv + WijUuiVvj

    #REPEAT
    for k in range(0,iter):
        # Rpuv
        Rpuv = calculate_Rpuv(n,m,a,b,W,U,V,z,p,q)

        #DIFuv = δuv (R̂uv − Ruv).
        DIFuv = calculate_DIFuv(R, Rpuv,mat)

        #z = z − η sumatoria1uv
        sum = np.sum(DIFuv)
        z = z - η * sum

        sumColDIF = np.sum(DIFuv, axis=1)
        #pu = pu − η X1 uv ,
        p = p - η*sumColDIF
        #Update U u according to Eq.(22);
        SWi  = np.sum(W, axis=0)
        SVi  = np.sum(V, axis=0)
        tem  = np.dot(SWi,SVi)
        temp = np.dot(sumColDIF,tem)
        temp = η*temp

        for u in range(0,n):
            for i in range(0,a):
                U[u,i]    = U[u,i] - temp[u]


        sumColDIFv = np.sum(DIFuv, axis=0)
        #qv = qv − η X1 uv ,
        q = q - η*sumColDIFv
        #Update U u according to Eq.(22);
        SWj = np.sum(W, axis=1)
        SVj = np.sum(V, axis=1)
        t   = np.dot(SWj,SVj)
        tmp = np.dot(sumColDIFv,t)

        for v in range(0,m):
            for j in range(0,b):
                V[v,j]    = V[v,j] - η*tmp[v]

            #V   = V - η*tmp

        #Update W according # W = np.random.random((a, b)) recordar sum
        SUui = np.sum(W, axis=0)
        SVvj = np.sum(W, axis=1)
        for i in range(0,a):
            for j in range(0,b):
                W[i,j] = W[i,j]-n*(sum*SUui[j]*SVvj[i])

    return (U,V)




In [ ]:
# Main

In [ ]:
rating = LeerDatos(dataset[1],',')
rating.shape
rating[100000,0]#Indice de fila 

In [ ]:
Rat = np.zeros((670,3953))
δuv = np.full((670,3953), 0)
for i in range(100000):
    Rat[rating[i,0],rating[i,1]] = rating[i,2]
    δuv[rating[i,0],rating[i,1]] = 1
    
Rat.shape #(670, 3953) Userx Item (n,m)


R = np.zeros((50,100))
for i in range(0,50):
    for j in range(0,100):
        R[i,j]=Rat[i,j]
        

# SINGULAR VALUE DECOMPOSITION
#svd = TruncatedSVD(n_components=600, n_iter=100, random_state=42)
#svd.fit(R)
#print(svd.singular_values_)
print(Rat)


In [ ]:
print(R)

In [ ]:
ηp = 0.1
itera = 3
U,V = Feature_Representation(R,ηp,itera)

In [ ]:
print()

In [ ]:
#Feature_Representation

In [ ]:
n = R.shape[0]
m = R.shape[1]

a = R.shape[1]
b = R.shape[0]
η = ηp#np.random.randn() 
#Randon Initialize z, p, q, U, V, W;
U = random_(n,a)#np.random.random((n,a))
V = random_(m,b)

z = np.random.randn()
p = np.random.randn(n)
q = np.random.randn(m)
W = random_(a,b)#np.random.random((a, b))

#Calculate 1 according to Eq.(24); 
    #Rpuv = z + pu + qv + WijUuiVvj
for k in range(0,1):
        # Rpuv
        #print("iteracion",k)
    
        Rpuv = calculate_Rpuv(n,m,a,b,W,U,V,z,p,q)
        
        #DIFuv = δuv (R̂uv − Ruv).
        DIFuv = calculate_DIFuv(R, Rpuv)
        #print("DIFuv")
        #print(DIFuv)

        #z = z − η sumatoria1uv
        sum = np.sum(DIFuv)
        z = z - η * sum

        sumColDIF = np.sum(DIFuv, axis=1)
        #pu = pu − η X1 uv ,
        p = p - η*sumColDIF
        #Update U u according to Eq.(22);
        SWi  = np.sum(W, axis=0)
        SVi  = np.sum(V, axis=0)
        tem  = np.dot(SWi,SVi)
        temp = np.dot(sumColDIF,tem)
        temp = η*temp
        
        sumColDIFv = np.sum(DIFuv, axis=0)
        #qv = qv − η X1 uv ,
        q = q - η*sumColDIFv
        #Update U u according to Eq.(22);
        SWj = np.sum(W, axis=1)
        SVj = np.sum(V, axis=1)
        t   = np.dot(SWj,SVj)
        tmp = np.dot(sumColDIFv,t)

        #for v in range(0,m):
            #for j in range(0,b):
                #V[v,j]    = V[v,j] - η*tmp[v]

            #V   = V - η*tmp

        #Update W according # W = np.random.random((a, b)) recordar sum
        SUui = np.sum(W, axis=0)
        SVvj = np.sum(W, axis=1)
        for i in range(0,a):
            for j in range(0,b):
                W[i,j] = W[i,j]-n*(sum*SUui[j]*SVvj[i])


In [ ]:
U.shape #(50, 100)
V.shape #(100, 50)


In [ ]:
# Tranpuesta
Vp = V.T
Vp.shape

In [ ]:
#Concatenar 
input_shape = U.shape[1]


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
#labelencoder_X_1 = LabelEncoder() 
#X[:,1] = labelencoder_X_1.fit_transform(X[:,1]) 
#labelencoder_X_2 = LabelEncoder() 
#X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [5]:
import keras 
from keras.models import Sequential 
from keras.layers import Dense

ModuleNotFoundError: No module named 'keras'

In [ ]:
# Create the model
model = Sequential()
model.add(Dense(a+b, input_shape=input_shape, activation='relu'))
model.add(Dense(a+b, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compiling Neural Network 
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])